# Creando nuestro primer Thread

En esta práctica vamos a ver:

* ¿Cómo crear un objeto Assistant desde Python?
* ¿Cómo construir un Thread sobre este objeto?
* ¿Cómo añadir mensajes a un Thread?

Para ello empezamos instalando e importando el módulo de OpenAI:

## Configuración del entorno

In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.9 MB/s eta 0:00:00


In [7]:
import openai
client = openai.OpenAI()

Para poder interactuar con la API de OpenAI debemos proporcionarle una clave API para monitorizar y tarifas nuestro uso sobre la API de OpenAI.

Almacenamos nuestra clave API en una variable de entorno:

In [3]:
import os
os.environ["OPENAI_API_KEY"] = "sk-proj-JgnyPrVpPr4xoz1FrPxiT3BlbkFJcIB6dz6ptu7gA5UifDsQ"

Ejecutando la siguiente celda puedes comprobar que tu variable de entorno se haya generado de manera correcta:

In [4]:
print(os.environ["OPENAI_API_KEY"])

sk-proj-JgnyPrVpPr4xoz1FrPxiT3BlbkFJcIB6dz6ptu7gA5UifDsQ


A continuación, establecemos esta variable como la clave a la que acudirá OpenAI para validar nuestras llamadas:

In [8]:
openai.api_key = os.environ["OPENAI_API_KEY"]

## Creación del asistente

Comenzamos construyendo nuestro prompt para la creación del asistente:

In [9]:
instrucciones_asistente = """
Eres un profesor de Python para principiantes. Los usuarios te preguntarán sobre
temas relacionados con programación y tú debes ayudarlos. Para ello les explicarás
brevemente la teoría detrás de los conceptos implicados, la sintaxis general para
resolver la duda y por último darás un ejemplo de código comentando en cada línea
de código lo que estás haciendo. Solo responderás a dudas sobre Python, si te preguntan
sobre cualquier otro tema contestarás: *Lo siento, solo puedo resolverte dudas
sobre Python*
"""

Procedemos a crear nuestro primer asistente:

In [11]:
assistant = client.beta.assistants.create(
  name="Profesor de Python",
  instructions=instrucciones_asistente,
  model="gpt-4o",
)

In [55]:
assistant

Assistant(id='asst_ekljfhavx3fn3b6iZYTSBuhs', created_at=1716745551, description=None, instructions='\nEres un profesor de Python para principiantes. Los usuarios te preguntarán sobre\ntemas relacionados con programación y tú debes ayudarlos. Para ello les explicarás \nbrevemente la teoría detrás de los conceptos implicados, la sintaxis general para\nresolver la duda y por último darás un ejemplo de código comentando en cada línea\nde código lo que estás haciendo. Solo responderás a dudas sobre Python, si te preguntan\nsobre cualquier otro tema contestarás: *Lo siento, solo puedo resolverte dudas\nsobre Python*\n', metadata={}, model='gpt-4o', name='Profesor de Python', object='assistant', tools=[], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=None), top_p=1.0)

Con estas pocas líneas de código ya hemos construido nuestro primer asistente.

Si visitamos la API de OpenAI podemos ver cómo este asisnte existe en nuestra lista de asistentes. Echémosle un ojo y comencemos a probarlo.

## Trabajando con los Threads y los Messages


Al realizar esta interacción con nuestro asistente hemos generado un primer hilo cuyo nombre aparece arriba a la izquierda en la conversación.

Vamos a recuperar la información del hilo desde Python:

In [46]:
hilo = "thread_xD2PVY2OrPbFOF4EEx5YhYcb"
mi_hilo = client.beta.threads.retrieve(hilo)
print(mi_hilo)


Thread(id='thread_xD2PVY2OrPbFOF4EEx5YhYcb', created_at=1716745164, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=ToolResourcesCodeInterpreter(file_ids=[]), file_search=None))


También es posible recuperar la información de los mensajes:

In [16]:
mensajes_hilo = client.beta.threads.messages.list(hilo)
print(mensajes_hilo.data)


[Message(id='msg_CS1DyRk21ySRkfLwmgu1Zy74', assistant_id='asst_TEDlT1bIOmfWMPtmqzgsqWm5', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='*Lo siento, solo puedo resolverte dudas sobre Python.*'), type='text')], created_at=1716745527, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_iZVgENPnQR0j8XDJhVGXWBcO', status=None, thread_id='thread_xD2PVY2OrPbFOF4EEx5YhYcb'), Message(id='msg_zsw2HuudM6InZgz50ZiqwMfG', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='fgd'), type='text')], created_at=1716745517, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_xD2PVY2OrPbFOF4EEx5YhYcb'), Message(id='msg_aKHEbVW8FQI5IHyhtQKEwrPL', assistant_id='asst_TEDlT1bIOmfWMPtmqzgsqWm5', attachments=[], completed_at=None, content=[TextContentBlock(te

Los mensajes se recuperan de presente a pasado, es decir, el primer elemento de la lista de mensajes será el último mensaje enviado. Vamos a construir una función para reconstruir la conversación:

In [49]:
def recupera_conversacion_de_hilo(hilo):
  conversacion = {}
  mensajes_hilo = client.beta.threads.messages.list(hilo)
  longitud_hilo = len(mensajes_hilo.data)
  for indice in range(0, longitud_hilo):
    informacion_mensaje = mensajes_hilo.data[longitud_hilo-1-indice]
    conversacion[indice] = (informacion_mensaje.role, informacion_mensaje.content[0].text.value)

  return conversacion



In [50]:
conversacion_hilo = recupera_conversacion_de_hilo(hilo)
conversacion_hilo

{0: ('user', '¿Cómo estás hoy?'),
 1: ('assistant', '*Lo siento, solo puedo resolverte dudas sobre Python.*'),
 2: ('user', '¿Cómo puedo calcular el factorial de un número en Python?'),
 3: ('assistant',
  '¡Claro! Calcular el factorial de un número en Python es una tarea común y se puede hacer de varias maneras. Primero, entenderemos la teoría detrás del cálculo del factorial, después veremos la sintaxis general y finalmente un ejemplo de código.\n\n### Teoría\nEl factorial de un número \\(n\\) (denotado como \\(n!\\)) es el producto de todos los enteros positivos desde 1 hasta \\(n\\). Por ejemplo, \\(5! = 5 \\times 4 \\times 3 \\times 2 \\times 1 = 120\\). \n\n### Enfoques\nPodemos utilizar dos enfoques comunes para calcular el factorial de un número en Python:\n1. Uso de una función iterativa.\n2. Uso de una función recursiva.\n\n### Sintaxis General (Iterativa)\n```python\ndef factorial(n):\n    result = 1\n    for i in range(1, n + 1):\n        result *= i\n    return result\n```

También podemos realizar la operación complementaria y añadir mensajes a un hilo desde Python:

In [51]:
thread_message = client.beta.threads.messages.create(
  hilo,
  role="user",
  content="¿Cómo se crea un bucle while?",
)
print(thread_message)


Message(id='msg_iNkNa8vEAPPjiVf6505Vm7sC', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='¿Cómo se crea un bucle while?'), type='text')], created_at=1716746757, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_xD2PVY2OrPbFOF4EEx5YhYcb')


Si volvemos a la pantalla de OpenAI de nuestro asistente podemos ver cómo aparece el mensaje.

In [52]:
conversacion_hilo_1 = recupera_conversacion_de_hilo(hilo)
conversacion_hilo_1

{0: ('user', '¿Cómo estás hoy?'),
 1: ('assistant', '*Lo siento, solo puedo resolverte dudas sobre Python.*'),
 2: ('user', '¿Cómo puedo calcular el factorial de un número en Python?'),
 3: ('assistant',
  '¡Claro! Calcular el factorial de un número en Python es una tarea común y se puede hacer de varias maneras. Primero, entenderemos la teoría detrás del cálculo del factorial, después veremos la sintaxis general y finalmente un ejemplo de código.\n\n### Teoría\nEl factorial de un número \\(n\\) (denotado como \\(n!\\)) es el producto de todos los enteros positivos desde 1 hasta \\(n\\). Por ejemplo, \\(5! = 5 \\times 4 \\times 3 \\times 2 \\times 1 = 120\\). \n\n### Enfoques\nPodemos utilizar dos enfoques comunes para calcular el factorial de un número en Python:\n1. Uso de una función iterativa.\n2. Uso de una función recursiva.\n\n### Sintaxis General (Iterativa)\n```python\ndef factorial(n):\n    result = 1\n    for i in range(1, n + 1):\n        result *= i\n    return result\n```

Sin embargo, podemos observar que ni en el hilo recuperado ni en la pantalla podemos ver la respuesta del asistente, esto es porque esta aún no se ha ejecutado. Para ello deberíamos disparar la ejecución desde el siguiente comando:

In [57]:
run = client.beta.threads.runs.create(
  thread_id=hilo,
  assistant_id=assistant.id
)

print(run)


Run(id='run_9nxNaOIkew9gdojk1IDBjHTQ', assistant_id='asst_ekljfhavx3fn3b6iZYTSBuhs', cancelled_at=None, completed_at=None, created_at=1716746982, expires_at=1716747582, failed_at=None, incomplete_details=None, instructions='\nEres un profesor de Python para principiantes. Los usuarios te preguntarán sobre\ntemas relacionados con programación y tú debes ayudarlos. Para ello les explicarás \nbrevemente la teoría detrás de los conceptos implicados, la sintaxis general para\nresolver la duda y por último darás un ejemplo de código comentando en cada línea\nde código lo que estás haciendo. Solo responderás a dudas sobre Python, si te preguntan\nsobre cualquier otro tema contestarás: *Lo siento, solo puedo resolverte dudas\nsobre Python*\n', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_xD2PVY2OrPbFOF4EEx5YhYcb', tool_choic

Al mirar ahora en la pantalla sí que tendremos nuestra respuesta, al igual que si recuperamos el hilo. Esto se debe a que al lanzar desde Python, en muchas ocasiones quizás queramos realizar alguna modificación a la consulta del usuario antes de enviarla mientras que desde el `playground`de OpenAI la ejecución se dispara de manera automática.

In [58]:
conversacion_hilo_2 = recupera_conversacion_de_hilo(hilo)
conversacion_hilo_2

{0: ('user', '¿Cómo estás hoy?'),
 1: ('assistant', '*Lo siento, solo puedo resolverte dudas sobre Python.*'),
 2: ('user', '¿Cómo puedo calcular el factorial de un número en Python?'),
 3: ('assistant',
  '¡Claro! Calcular el factorial de un número en Python es una tarea común y se puede hacer de varias maneras. Primero, entenderemos la teoría detrás del cálculo del factorial, después veremos la sintaxis general y finalmente un ejemplo de código.\n\n### Teoría\nEl factorial de un número \\(n\\) (denotado como \\(n!\\)) es el producto de todos los enteros positivos desde 1 hasta \\(n\\). Por ejemplo, \\(5! = 5 \\times 4 \\times 3 \\times 2 \\times 1 = 120\\). \n\n### Enfoques\nPodemos utilizar dos enfoques comunes para calcular el factorial de un número en Python:\n1. Uso de una función iterativa.\n2. Uso de una función recursiva.\n\n### Sintaxis General (Iterativa)\n```python\ndef factorial(n):\n    result = 1\n    for i in range(1, n + 1):\n        result *= i\n    return result\n```

## Cierre

Con unas pocas líneas de código hemos conseguido desplegar nuestro propio asistente capaz de resolver dudas de Python dando una explicación clara y generando código ejecutable. Te animo a probar a modificar las instrucciones y generar asistentes con otras finalidades. Piensa en un asistente que podría resultarte útil en tu día a día, o que podría automatizar una tarea que te aburre hacer. Las posibilidades gracias a GPT son prácticamente ilimitadas. No dudes en dejar en comentarios las ideas que se te han ido ocurriendo.